select \* on the SalesLT.Address table from the AdventureWorks Table.  The key field for the row level security is <mark>Country</mark>

In [1]:
select Top 100 * from SalesLT.Address


(0 rows affected)

Total execution time: 00:00:00.579

AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate


Create User Logins on the Master Database that we will use to test Row Level Security

In [10]:
-- Creates the logins in the master db with password 'Bazinga!1234'.  
CREATE LOGIN vendorusa 
    WITH PASSWORD = 'Bazinga!1234';  

CREATE LOGIN vendorcan
    WITH PASSWORD = 'Bazinga!1234';  
GO  
  
CREATE LOGIN vendormgr
    WITH PASSWORD = 'Bazinga!1234';  
GO  

: Msg 15025, Level 16, State 1, Line 2
The server principal 'vendorusa' already exists.

: Msg 15025, Level 16, State 1, Line 5
The server principal 'vendorcan' already exists.

Commands completed successfully.

Total execution time: 00:00:00.097

Creates a database user for the logins created above on the AdventureWorks database.

In [30]:
CREATE USER vendorusa FOR LOGIN vendorusa; 
CREATE USER vendorcan FOR LOGIN vendorcan;
CREATE USER vendormgr FOR LOGIN vendormgr;
GO

Commands completed successfully.

Total execution time: 00:00:00.067

Grant read access on the table to each of the users.  This is the only access they have to the database.

In [13]:
GRANT SELECT ON SalesLT.Address TO VendorUSA;  
GRANT SELECT ON SalesLT.Address TO VendorCAN; 
GRANT SELECT ON SalesLT.Address TO VendorMGR;
GO

Commands completed successfully.

Total execution time: 00:00:00.067

Create a table to hold the lookup data – in this case a relationship between Address.Country and Vendor

In [7]:
CREATE TABLE SalesLT.Lk_Country_Vendor
  ( Vendor sysname,
    CountryRegion varchar(50)
  ) ;

Commands completed successfully.

Total execution time: 00:00:00.602

Populate the lookup table with sample data, linking Country to vendor.

In [26]:
--INSERT INTO SalesLT.Lk_Country_Vendor VALUES ('dbo', 'United Kingdom');

delete from SalesLT.Lk_Country_Vendor where COUNTRYRegion = 'United Kingdon'



(1 row affected)

Total execution time: 00:00:00.271

In [2]:
-- View the 2 rows in the table
SELECT * FROM SalesLT.Lk_Country_Vendor;

(4 rows affected)

Total execution time: 00:00:00.186

Vendor,CountryRegion
VendorUSA,United States
VendorCAN,Canada
Jack.Bender@microsoft.com,United Kingdom
dbo,United Kingdom


In [4]:
select SYSTEM_USER

(1 row affected)

Total execution time: 00:00:00.068

(No column name)
synapseadmin


Create a new schema, and an inline table-valued function. The function returns 1 when a user queries the fact table Address and the Vendor column of the table Lk_Country_Vendor is the same as the user executing the query (@Vendor = USER_NAME()) when joined to the fact table on the CountryRegion column, or if the user executing the query is the Manager user (USER_NAME() = 'Manager').

In [37]:
CREATE SCHEMA Security ;
GO
CREATE FUNCTION Security.fn_securitypredicate
         (@CountryRegion AS varchar(50))
RETURNS TABLE
WITH SCHEMABINDING
AS
           RETURN ( SELECT 1 as Result
                     FROM SalesLT.Address A
            INNER JOIN SalesLT.Lk_Country_Vendor L
                     ON A.CountryRegion = L.CountryRegion
            WHERE ( A.CountryRegion = @CountryRegion
                    AND L.Vendor = SYSTEM_USER )
                 OR SYSTEM_USER = 'vendormgr'
                   ) ;

Commands completed successfully.

Total execution time: 00:00:00.080

In [36]:
--DROP FUNCTION Security.fn_securitypredicate

Commands completed successfully.

Total execution time: 00:00:00.203

Create a security policy adding the function as a filter predicate. The state must be set to ON to enable the policy  
<mark>We specify the VendorID field on the yellow 2019 table as that is the field our FILTER PREDICATE will use to filter data based on the logged in user.</mark>

In [38]:
CREATE SECURITY POLICY VendorFilter  
ADD FILTER PREDICATE Security.fn_securitypredicate(CountryRegion)
ON SalesLT.Address
WITH (STATE = ON);  
GO

Commands completed successfully.

Total execution time: 00:00:00.094

In [32]:
DROP SECURITY POLICY VendorFilter  

Commands completed successfully.

Total execution time: 00:00:00.071

Allow SELECT permissions to the fn_securitypredicate function

In [39]:
GRANT SELECT ON Security.fn_securitypredicate TO VendorUSA;  
GRANT SELECT ON Security.fn_securitypredicate TO VendorCAN; 
GRANT SELECT ON Security.fn_securitypredicate TO VendorMGR; 

Commands completed successfully.

Total execution time: 00:00:00.130

Now test the filtering predicate, by selected from the table as each user.  
<mark>This may cause data explorer to error and you may need to perform this in SQL Server Management Studio or Synapse Workspace as when Data Explorer errors, you must close it out and restart it again.</mark>

In [5]:
EXECUTE AS USER = 'VendorUSA';  
SELECT * FROM [SalesLT].Address;
REVERT;  



: Msg 40529, Level 16, State 1, Line 1
Built-in function 'SUSER_SNAME' in impersonation context is not supported in this version of SQL Server.

Total execution time: 00:00:00.122

In [19]:
EXECUTE AS USER = 'VendorCAN';  
SELECT * FROM [SalesLT].Address;
REVERT;  

 

(115 rows affected)

Total execution time: 00:00:00.608

AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
488,45259 Canada Way,NULL,Burnaby,British Columbia,Canada,V5G 4S4,6D2D6846-3F17-474E-8D31-AD3A59C0CEB0,2007-08-01 00:00:00.0000000
514,63 West Beaver Creek,NULL,Richmond Hill,Ontario,Canada,L4E 3M5,AD60DBA9-3B01-47E9-BB5E-DF94D0579F14,2006-09-01 00:00:00.0000000
511,"Bureau 9900 825, Avenue Querbes",NULL,Outremont,Quebec,Canada,H1Y 2G5,EF10CF20-A540-4D5C-BEFD-1B48B7D48402,2006-12-01 00:00:00.0000000
519,2500-622 5th Ave Sw,NULL,Calgary,Alberta,Canada,T2P 2G8,13754965-6A09-4343-968C-8699C27FDFAA,2006-08-01 00:00:00.0000000
482,2500 University Avenue,NULL,Toronto,Ontario,Canada,M4B 1V5,197A1735-E89E-4F15-A5F7-FEB98B369094,2005-07-01 00:00:00.0000000
507,975 Princess Street,NULL,Kingston,Ontario,Canada,7L,D7849F5F-C042-4FCE-8F6B-BA5CD8708E10,2007-09-01 00:00:00.0000000
462,630 University Avenue,NULL,Toronto,Ontario,Canada,M4B 1V7,27D5BDAD-647D-4A56-859A-BC6A82AF7DCD,2005-08-01 00:00:00.0000000
500,5700 Explorer Drive,NULL,Mississauga,Ontario,Canada,L4W 5J3,F6FED7FA-39D9-4A60-A1F5-1622489E55FD,2005-08-01 00:00:00.0000000
506,255117 - 101 Avenue,NULL,Surrey,British Columbia,Canada,V3T 4W3,22895B47-6086-4550-B437-A75DBEDFE1D9,2005-08-01 00:00:00.0000000
526,251420a 170th Street,NULL,Edmonton,Alberta,Canada,T5,476D80AE-69A0-480D-9DF3-8FD0C1095A21,2006-07-01 00:00:00.0000000


In [20]:
EXECUTE AS USER = 'VendorMGR';  
SELECT * FROM [SalesLT].Address;
REVERT;  



(450 rows affected)

Total execution time: 00:00:00.504

AddressID,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,rowguid,ModifiedDate
826,Kensington Valley Shops,NULL,Howell,Michigan,United States,48843,C2D04D48-EF2B-4349-877B-EA6DF9895079,2006-07-01 00:00:00.0000000
1031,Natomas Marketplace,NULL,Sacramento,California,United States,95814,27AF5ED8-D33A-4FB2-9C55-D5CDD5017CD4,2006-08-01 00:00:00.0000000
841,Washington Square,NULL,Portland,Oregon,United States,97205,7B5AE329-FB7C-4A2C-B33C-827920ACACDC,2005-11-01 00:00:00.0000000
519,2500-622 5th Ave Sw,NULL,Calgary,Alberta,Canada,T2P 2G8,13754965-6A09-4343-968C-8699C27FDFAA,2006-08-01 00:00:00.0000000
482,2500 University Avenue,NULL,Toronto,Ontario,Canada,M4B 1V5,197A1735-E89E-4F15-A5F7-FEB98B369094,2005-07-01 00:00:00.0000000
652,Wymbush,NULL,Milton Keynes,England,United Kingdom,MK8 8DF,54E20963-B0E9-41CF-AB7E-D50459A3325C,2006-09-01 00:00:00.0000000
597,99450 Highway 59 North,NULL,Humble,Texas,United States,77338,3BD8466E-2A52-4661-8460-50736409EE8A,2006-08-01 00:00:00.0000000
500,5700 Explorer Drive,NULL,Mississauga,Ontario,Canada,L4W 5J3,F6FED7FA-39D9-4A60-A1F5-1622489E55FD,2005-08-01 00:00:00.0000000
832,2251 Elliot Avenue,NULL,Seattle,Washington,United States,98104,6D6A8E1A-FFD8-4D2D-A70D-88467049A880,2005-08-01 00:00:00.0000000
593,2345 North Freeway,NULL,Houston,Texas,United States,77003,74E5959F-4C5F-4BDA-ABCF-429762E1670D,2005-08-01 00:00:00.0000000


In [21]:
GRANT SELECT ON Security.fn_securitypredicate TO [Jack.Bender@microsoft.com];

: Msg 15151, Level 16, State 1, Line 1
Cannot find the user 'Jack.Bender@microsoft.com', because it does not exist or you do not have permission.

Total execution time: 00:00:00.053